# Larval TagSeq Processing
This notebook contains the code used to process the TagSeq data modified from Dr. Misha Matz's code (https://github.com/z0on/tag-based_RNAseq/blob/master/tagSeq_processing_README.txt) for running on the Texas State University HPC LEAP2. This code requires you to download the scripts from Dr. Matz's tag-based_RNAseq repository on GitHub and import it to the HPC. I store those scripts in my home directory and therefore, all of my code below will write the paths to each script starting from the home directory.

The sequence data used in this code was from a project using *Astrangia poculata* larvae to look at the response of larvae to both probiotic and pathogenic bacteria and are published in the manuscript below.

#### Manuscript Reference
Borbee, E.M., Changsut, I.V., Bernabe, K., Schickle, A., Nelson, D., Sharp, K.H., and Fuess, L.E. (2025) Coral larvae have unique transcriptomic responses to pathogenic and probiotic bacteria. *In prep*.

## Contents
1. [Required programs](#1)
2. [Quality assessment of reads](#2)
3. [Adapter trimming, quality filtering, and deduplicating reads](#3)
4. [Mapping reads to reference transcriptome](#4)
5. [Generating counts per transcript](#5)
6. [Compiling count matrix](#7)

<a id="1"></a>
### 1. Required programs
The programs you need to run this script include `fastqc`, `cutadapt`, `bowtie2`, and `samtools`. You will also need to download Dr. Matz's tag_based_RNAseq git repository to your computer so that the scripts contained in that repository are accessible from where you are running your code. Below I provide an example of a miniconda command I used to install `fastqc`, `cutadapt`, `bowtie2`, and `samtools`.

In [ ]:
conda install bioconda::cutadapt

<a id=2></a>
### 2. Quality assessment of reads
Raw sequencing reads were first quality assessed using the `fastqc` command below. We used this to make sure we had high quality reads across all samples and identify trimming parameters for the next step.

In [ ]:
fastqc *.fastq.gz

<a id=3></a>
### 3. Adapter trimming, quality filtering, and deduplicating reads
In this step we first use cutadapt to remove adapter sequences from reads and remove short or low quality reads and then use a custom perl script from Dr. Matz's GitHub to deduplicate the sequences. The code below is the slurm command that was submitted to the LEAP2 job manager to complete this step.

In [ ]:
#!/bin/bash
#SBATCH --job-name=cutadapt
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=shared
#SBATCH --mem=50G
#SBATCH --array=1-9%10
#SBATCH --mail-type=end
#SBATCH --mail-user=eborbee@txstate.edu
#SBATCH -o cutadapt_%j.out
#SBATCH -e cutadapt_%j.err

~/tag-based_RNAseq/tagseq_clipper.pl raw_data/EB00${SLURM_ARRAY_TASK_ID}.fastq | cutadapt - -a AAAAAAAA -a AGATCGG -q 15 -m 25 -o EB00${SLURM_ARRAY_TASK_ID}_trimmed.fastq

This step filtered our initial 40,118,656 reads to 9,513,928 reads (24% of the dataset).

### 4. Mapping reads to reference transcriptome
Here we map the reads from our samples to an *Astrangia poculata* reference transcriptome. Below is first a command for building an index from the reference transcriptome and then the slurm script that was submitted to our job manager. Our reference transcriptome file came from the publication cited below.

#### Reference transcriptome paper
Changsut, I. V., Borbee, E. M., Womack, H. R., Shickle, A., Sharp, K. H., & Fuess, L. E. (2024). Photosymbiont density is correlated with constitutive and induced immunity in the facultatively symbiotic coral, Astrangia poculata. Integrative and Comparative Biology, 64(5), 1278-1290.

In [ ]:
bowtie2-build transcriptome.fasta transcriptome.fasta

In [ ]:
#!/bin/bash
#SBATCH --job-name=maps
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=himem
#SBATCH --mem=50G
#SBATCH --array=1-9%10
#SBATCH --open-mode=append
#SBATCH --mail-type=end
#SBATCH --mail-user=loe8@txstate.edu
#SBATCH -o maps_%A.out
#SBATCH -e maps_%A.err

bowtie2 --local -x /home/loe8/AstrangiaTranscriptome_042622/filtered-final-transcriptome.fa -U EB00${SLURM_ARRAY_TASK_ID}_trimmed.fastq -S EB00${SLURM_ARRAY_TASK_ID}_trimmed.fastq.sam --no-hd --no-sq --no-unal -k 5

Next we ran a grep command to check the mapping rates of our transcripts to the reference transcriptome.

In [ ]:
grep "overall alignment rate" maps_5573.err

The mapping rates for our data to the reference were as follows:

`28.08% overall alignment rate
36.81% overall alignment rate
35.39% overall alignment rate
28.88% overall alignment rate
36.25% overall alignment rate
29.49% overall alignment rate
33.73% overall alignment rate
31.57% overall alignment rate
31.38% overall alignment rate
26.14% overall alignment rate
31.07% overall alignment rate
32.83% overall alignment rate
33.25% overall alignment rate
29.34% overall alignment rate
33.14% overall alignment rate`

<a id=5></a>
### Generating counts per transcript
After mapping our transcripts to the reference transcriptome we then generated counts of each transcript per sample using `samtools`. Below is the slurm script submitted to the job manager used to do this. 

In [ ]:
#!/bin/bash
#SBATCH --job-name=samcount
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=himem
#SBATCH --mem=50G
#SBATCH --array=1-9%10
#SBATCH --open-mode=append
#SBATCH --mail-type=end
#SBATCH --mail-user=loe8@txstate.edu
#SBATCH -o samcount_%A.out
#SBATCH -e samcount_%A.err

~/tag-based_RNAseq/samcount.pl EB00${SLURM_ARRAY_TASK_ID}_trimmed.fastq.sam transcriptome_seq2gene.tab aligner=bowtie2 >EB00${SLURM_ARRAY_TASK_ID}_trimmed.fastq.sam.counts

<a id=6></a>
### Compiling count matrix
After generating the count files we then combined the individual sample files into a single count matrix using a custom perl script from Dr. Matz's GitHub. The command we used to run that is provided below.

In [ ]:
~/tag-based_RNAseq/expression_compiler.pl *.sam.counts > allcounts.txt